In [27]:
from utils.data_preprocess import *
from models import CDT_1D_model, MLP_model
import pandas as pd
import numpy as np
import tensorflow as tf
import talib as ta

In [26]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')
raw_data = DataLabeling(df, 24)
train_df, val_df, test_df = train_val_test_split(raw_data.TI_data)

In [30]:
Data = DataPreprocess(train_df, val_df, test_df, window_size=24, label_size=1, label_columns=['Label'], shift=0)
Data.train

<MapDataset shapes: ((None, 24, 43), (None, 1, 1)), types: (tf.float32, tf.float32)>

In [33]:
model = MLP_model(24, 43)